In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-cp37-none-any.whl size=16443 sha256=1b327f6ca455be23275707cf97cf9fe857e685c338614a4d18b28394691c7d8e
  Stored in directory: /root/.cache/pip/wheels/84/27/aa/c46d23c4e8cc72d41283862b1437e0b3ad318417e8ed7d5921
Successfully built efficientnet-pytorch


In [3]:
root_dir = '/content/drive/MyDrive/Models_exp2.5'

In [4]:
import os
import multiprocessing
import numpy as np
import cv2
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms

from efficientnet_pytorch import EfficientNet

In [5]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  

device = torch.device(dev)
print(device)

cuda:0


In [6]:
!nvidia-smi

Thu Apr 29 04:06:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
#https://towardsdatascience.com/implementing-the-new-state-of-the-art-mish-activation-with-2-lines-of-code-in-pytorch-e7ef438a5ee7
def mish(x): 
  return (x * torch.tanh(nn.functional.softplus(x)))

In [8]:
#https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/
class Model_3d(nn.Module):
    '''
    A model to use on 3d objects
    '''
    def __init__(self, fc_nodes, no_views, no_classes):
      super(Model_3d, self).__init__()

      #https://www.kaggle.com/ateplyuk/pytorch-efficientnet
      self.resnet = EfficientNet.from_pretrained('efficientnet-b6')

      # Unfreeze model weights
      for param in self.resnet.parameters():
          param.requires_grad = True



      num_features = 1000

      self.fcs = nn.ModuleList()

      for idx in range(no_views):
        self.fcs.append(nn.Linear(num_features, fc_nodes))

      first_dense_count = fc_nodes * no_views

      self.dense_1 = nn.Linear(first_dense_count, no_classes)

    
    def forward(self, x):
      features = []

      for i, fc in enumerate(self.fcs):
        sample = x[:, i, ...]
        r_out = self.resnet(sample)

        fc_out = fc(r_out)
        features.append(fc_out)
      
      stack = torch.stack(features, 1)

      reshaped = stack.view([x.shape[0], -1])

      x = self.dense_1(reshaped)


      return torch.nn.functional.softmax(x)

In [9]:
class Date_3d_Cached(torch.utils.data.Dataset):
    def __init__(self, x, y):
      self.x = torch.from_numpy(x)
      self.y = torch.from_numpy(y)
    
    def __len__(self):
      return self.x.shape[0]

    def __getitem__(self, idx):
      x = self.x[idx] /255.0
      return x, self.y[idx]

In [10]:
views = ['bottom', 'side_1', 'side_2', 'side_3', 'side_4', 'top', 'top_1', 'top_2', 'top_3', 'top_4', 'bot_1', 'bot_2', 'bot_3', 'bot_4']
x = np.load('/content/drive/MyDrive/Cache/x_compressed.npz')['arr_0']
y = np.load('/content/drive/MyDrive/Cache/y.npy')

In [11]:
model = Model_3d(1024, len(views), 10)
model.to(device)
print(device)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b6-c76e70fd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b6-c76e70fd.pth



Loaded pretrained weights for efficientnet-b6
cuda:0


In [12]:
training_data = Date_3d_Cached(x, y)
#90/10 train val split
train_len = int(len(training_data) * .9)
val_len = len(training_data) - train_len

In [13]:
train_set, val_set = torch.utils.data.random_split(training_data, [train_len, val_len],torch.Generator().manual_seed(42))

In [14]:
batch_size = 2

In [15]:
training_loader = torch.utils.data.DataLoader(train_set, batch_size= batch_size, shuffle=True, num_workers=os.cpu_count())
val_loader = torch.utils.data.DataLoader(val_set, batch_size= batch_size, shuffle=True, num_workers=os.cpu_count())

In [16]:
print(os.cpu_count())
print(train_len)
print(val_len)

4
3591
400


In [17]:
#https://medium.com/analytics-vidhya/saving-and-loading-your-model-to-resume-training-in-pytorch-cb687352fa61
def save_ckp(state, model_path):
    torch.save(state, model_path)

In [18]:
#https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:


if not os.path.exists(root_dir):
    os.makedirs(root_dir)


low_val_loss = 30000000
best_acc = -1
epoch = 0

val_loss_path = None
acc_path = None
#https://stackoverflow.com/questions/8078330/csv-writing-within-loop
import csv
with open(os.path.join(root_dir, 'logs.csv'), 'w') as file_csv:
  writer=csv.writer(file_csv, delimiter=',',lineterminator='\n',)
  writer.writerow(['epoch', 'loss', 'acc', 'val_loss', 'val_acc'])


  for epoch in range(1, 101):
      row = [epoch]
      is_best = False
      running_loss = 0.0
      correct = 0
      total = 0

      model.train()

      t0 = time.time()
      for i, data in enumerate(training_loader):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data

          # for this_view in range(inputs.shape[1]):
          #   if np.random.randint(0, 4) == 0:
          #     inputs[:, this_view, ...] = 1.0

          inputs = inputs.to(device)
          labels = labels.to(device)

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # print statistics
          correct += (outputs.argmax(1) == labels).float().sum() #https://stackoverflow.com/questions/51503851/calculate-the-accuracy-every-epoch-in-pytorch
          total += labels.shape[0]
          running_loss += loss.item()

      print('epoch', epoch)
      
      print('{} seconds'.format(time.time() - t0))
      print('loss', running_loss)
      row.append(running_loss)
      running_loss = 0.0

      accuracy = 100 * correct / total
      print("Accuracy = {}".format(accuracy))
      row.append(torch.IntTensor.item(accuracy))
      correct = 0
      total= 0

      running_loss = 0.0
      correct = 0
      total = 0

      optimizer.zero_grad()
      model.eval()
      t0 = time.time()
      with torch.no_grad():
          for i, data in enumerate(val_loader):
              
              # get the inputs; data is a list of [inputs, labels]
              inputs, labels = data
              inputs = inputs.to(device)
              labels = labels.to(device)            


              # forward + backward + optimize
              outputs = model(inputs)
              loss = criterion(outputs, labels)

              correct += (outputs.argmax(1) == labels).float().sum() #https://stackoverflow.com/questions/51503851/calculate-the-accuracy-every-epoch-in-pytorch
              total += labels.shape[0]
              running_loss += loss.item()

      print('{} seconds'.format(time.time() - t0))
      print('val loss', running_loss)
      row.append(running_loss)
      

      accuracy = 100 * correct / total
      print(" Val Accuracy = {}".format(accuracy))
      row.append(torch.IntTensor.item(accuracy))

      model_name = 'ep_' + str(epoch) + '_loss_' + str(running_loss) + '_acc_' + str(accuracy) + '.pt'
      full_model_path = os.path.join(root_dir, model_name)
      checkpoint = {
          'epoch': epoch + 1,
          'state_dict': model.state_dict(),
          'optimizer': optimizer.state_dict()
      }
      if running_loss < low_val_loss:
        if val_loss_path and os.path.exists(val_loss_path):
          open(val_loss_path, 'w').close() #overwrite and make the file blank instead - ref: https://stackoverflow.com/a/4914288/3553367
          os.remove(val_loss_path)
        val_loss_path = full_model_path.replace('.pt', 'val_loss_best.pt')
        save_ckp(model, val_loss_path)
        low_val_loss = running_loss
        print('new low loss')

      if accuracy > best_acc:
        if acc_path and os.path.exists(acc_path):
          open(acc_path, 'w').close() #overwrite and make the file blank instead - ref: https://stackoverflow.com/a/4914288/3553367
          os.remove(acc_path)
        acc_path = full_model_path.replace('.pt', 'acc_best.pt')
        save_ckp(model, acc_path)
        best_acc = accuracy
        print('new best acc')
      full_model_path = os.path.join(root_dir, 'last.pt')
      save_ckp(checkpoint, full_model_path)
          

      

      correct = 0
      total= 0
      running_loss = 0.0
      writer.writerow(row)
  print('Finished Training')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 1
2863.560446739197 seconds
loss 3179.6136000156403
Accuracy = 70.50961303710938
92.9740560054779 seconds
val loss 341.93410789966583
 Val Accuracy = 84.25
new low loss
new best acc
epoch 2
2871.999974489212 seconds
loss 2947.7740285396576
Accuracy = 82.62322998046875
92.69584131240845 seconds
val loss 336.807346701622
 Val Accuracy = 83.25
new low loss
epoch 3
2881.3288033008575 seconds
loss 2884.3004355430603
Accuracy = 86.13200378417969
92.9067370891571 seconds
val loss 319.7000457048416
 Val Accuracy = 88.5
new low loss
new best acc
epoch 4
2886.530862092972 seconds
loss 2842.310143828392
Accuracy = 88.3041000366211
92.98687553405762 seconds
val loss 317.6182872056961
 Val Accuracy = 90.5
new low loss
new best acc
epoch 5
2918.251447200775 seconds
loss 2777.956920146942
Accuracy = 91.9799575805664
93.020583152771 seconds
val loss 314.5653438568115
 Val Accuracy = 91.0
new low loss
new best acc
epoch 6
2917.4682290554047 seconds
loss 2748.8635001182556
Accuracy = 93.5951004028

In [ ]:
import pandas as pd
     
# dictionary of lists  
this_dict = {'training loss': training_loss_list, 'training acc': training_acc_list, 'val loss': val_loss_list, 'val acc':val_acc_list}  
       
df = pd.DataFrame(this_dict) 
    
# saving the dataframe 
df.to_csv(os.path.join(root_dir, 'logs.csv'))

In [ ]:
print('saved csv')

In [ ]:
print(row)

In [ ]:
print(inputs.shape)

In [19]:
cp = torch.load(os.path.join(root_dir, 'last.pt'))

In [20]:
      # checkpoint = {
      #     'epoch': epoch + 1,
      #     'state_dict': model.state_dict(),
      #     'optimizer': optimizer.state_dict()
      # }

model.load_state_dict(cp['state_dict'])
optimizer.load_state_dict(cp['optimizer'])
last_epoch_done = cp['epoch'] - 1

In [ ]:
#continue training


if not os.path.exists(root_dir):
    os.makedirs(root_dir)


low_val_loss = 30000000
best_acc = -1
epoch = 0

val_loss_path = None
acc_path = None
#https://stackoverflow.com/questions/8078330/csv-writing-within-loop
import csv
with open(os.path.join(root_dir, 'logs2.csv'), 'w') as file_csv:
  writer=csv.writer(file_csv, delimiter=',',lineterminator='\n',)
  writer.writerow(['epoch', 'loss', 'acc', 'val_loss', 'val_acc'])


  for epoch in range(last_epoch_done, 101):
      row = [epoch]
      is_best = False
      running_loss = 0.0
      correct = 0
      total = 0

      model.train()

      t0 = time.time()
      for i, data in enumerate(training_loader):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data

          # for this_view in range(inputs.shape[1]):
          #   if np.random.randint(0, 4) == 0:
          #     inputs[:, this_view, ...] = 1.0

          inputs = inputs.to(device)
          labels = labels.to(device)

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # print statistics
          correct += (outputs.argmax(1) == labels).float().sum() #https://stackoverflow.com/questions/51503851/calculate-the-accuracy-every-epoch-in-pytorch
          total += labels.shape[0]
          running_loss += loss.item()

      print('epoch', epoch)
      
      print('{} seconds'.format(time.time() - t0))
      print('loss', running_loss)
      row.append(running_loss)
      running_loss = 0.0

      accuracy = 100 * correct / total
      print("Accuracy = {}".format(accuracy))
      row.append(torch.IntTensor.item(accuracy))
      correct = 0
      total= 0

      running_loss = 0.0
      correct = 0
      total = 0

      optimizer.zero_grad()
      model.eval()
      t0 = time.time()
      with torch.no_grad():
          for i, data in enumerate(val_loader):
              
              # get the inputs; data is a list of [inputs, labels]
              inputs, labels = data
              inputs = inputs.to(device)
              labels = labels.to(device)            


              # forward + backward + optimize
              outputs = model(inputs)
              loss = criterion(outputs, labels)

              correct += (outputs.argmax(1) == labels).float().sum() #https://stackoverflow.com/questions/51503851/calculate-the-accuracy-every-epoch-in-pytorch
              total += labels.shape[0]
              running_loss += loss.item()

      print('{} seconds'.format(time.time() - t0))
      print('val loss', running_loss)
      row.append(running_loss)
      

      accuracy = 100 * correct / total
      print(" Val Accuracy = {}".format(accuracy))
      row.append(torch.IntTensor.item(accuracy))

      model_name = 'ep_' + str(epoch) + '_loss_' + str(running_loss) + '_acc_' + str(accuracy) + '.pt'
      full_model_path = os.path.join(root_dir, model_name)
      checkpoint = {
          'epoch': epoch + 1,
          'state_dict': model.state_dict(),
          'optimizer': optimizer.state_dict()
      }
      if running_loss < low_val_loss:
        if val_loss_path and os.path.exists(val_loss_path):
          open(val_loss_path, 'w').close() #overwrite and make the file blank instead - ref: https://stackoverflow.com/a/4914288/3553367
          os.remove(val_loss_path)
        val_loss_path = full_model_path.replace('.pt', 'val_loss_best.pt')
        save_ckp(model, val_loss_path)
        low_val_loss = running_loss
        print('new low loss')

      if accuracy > best_acc:
        if acc_path and os.path.exists(acc_path):
          open(acc_path, 'w').close() #overwrite and make the file blank instead - ref: https://stackoverflow.com/a/4914288/3553367
          os.remove(acc_path)
        acc_path = full_model_path.replace('.pt', 'acc_best.pt')
        save_ckp(model, acc_path)
        best_acc = accuracy
        print('new best acc')
      full_model_path = os.path.join(root_dir, 'last.pt')
      save_ckp(checkpoint, full_model_path)
          

      

      correct = 0
      total= 0
      running_loss = 0.0
      writer.writerow(row)
  print('Finished Training')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


epoch 18
2858.061471939087 seconds
loss 2644.2417652606964
Accuracy = 98.91395568847656
93.03091669082642 seconds
val loss 311.39091753959656
 Val Accuracy = 92.25
new low loss
new best acc
epoch 19
2862.072464942932 seconds
loss 2640.6770712137222
Accuracy = 99.16458129882812
93.01186394691467 seconds
val loss 313.2772363424301
 Val Accuracy = 93.25
new best acc
epoch 20
2862.2704916000366 seconds
loss 2639.5203055143356
Accuracy = 99.27597045898438
92.86307501792908 seconds
val loss 313.46524310112
 Val Accuracy = 92.0
epoch 21
2866.0504529476166 seconds
loss 2639.268961429596
Accuracy = 99.27597045898438
92.75195813179016 seconds
val loss 314.67598056793213
 Val Accuracy = 91.5
epoch 22
2869.610503435135 seconds
loss 2641.336545228958
Accuracy = 99.08103942871094
92.77650666236877 seconds
val loss 314.7834873199463
 Val Accuracy = 92.0
epoch 23
2868.2953600883484 seconds
loss 2640.901132106781
Accuracy = 99.13673400878906
92.85184073448181 seconds
val loss 318.8367986679077
 Val Acc